In [ ]:
from typing import Optional, Literal, Callable, Sequence, Any
from pathlib import Path
from functools import partial
from multiprocessing import Pool
from tqdm.notebook import tqdm
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import pydicom
from PIL import Image
import cv2
cv2.setNumThreads(0)
from catboost import CatBoostClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

import lightning.pytorch as pl
from torchmetrics.classification import (
    MultilabelAUROC, MultilabelAveragePrecision
)
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.utilities.types import OptimizerLRScheduler

from torchvision.models.resnet import resnet50

from moonshot.model import moonshot
from moonshot.image_processor import ImageProcessor
from moonshot.utils import eval_mode


DEVICE = torch.device('cuda')
NUM_WORKERS = 16  # поменяйте на кол-во доступных CPU

# укажите полный или относительный путь до файла с весами модели Moonshot
MOONSHOT_MODEL_FILEPATH = Path('../vision_model.pt')  # путь до модели, если она лежит в корне репозитория

In [ ]:
VINDRCXR_DIRPATH = Path('D:/ds_clav_fracture_train_good/block_0000_anon/')

In [ ]:
dicom_filepath = random.choice(list((VINDRCXR_DIRPATH / 'train').iterdir()))
dicom = pydicom.dcmread(dicom_filepath)
image = ImageProcessor.preprocess_dicom(dicom)
image.show()

In [ ]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
